In [1]:
import os
from pathlib import Path

import pandas as pd
REPO_DIR = Path(os.path.realpath("")).parent

In [2]:
df = pd.read_parquet(REPO_DIR / "data/hotel_bookings.parquet") 

In [3]:
df.head()

,hotel,is_canceled,lead_time,arrival_date_year,arrival_date_month,arrival_date_week_number,arrival_date_day_of_month,stays_in_weekend_nights,stays_in_week_nights,adults,...,deposit_type,agent,company,days_in_waiting_list,customer_type,adr,required_car_parking_spaces,total_of_special_requests,reservation_status,reservation_status_date
0,Resort Hotel,0,342,2015,July,27,1,0,0,2,...,No Deposit,NaN,NaN,0,Transient,0.0,0,0,Check-Out,2015-07-01
1,Resort Hotel,0,737,2015,July,27,1,0,0,2,...,No Deposit,NaN,NaN,0,Transient,0.0,0,0,Check-Out,2015-07-01
2,Resort Hotel,0,7,2015,July,27,1,0,1,1,...,No Deposit,NaN,NaN,0,Transient,75.0,0,0,Check-Out,2015-07-02
3,Resort Hotel,0,13,2015,July,27,1,0,1,1,...,No Deposit,304.0,NaN,0,Transient,75.0,0,0,Check-Out,2015-07-02
4,Resort Hotel,0,14,2015,July,27,1,0,2,2,...,No Deposit,240.0,NaN,0,Transient,98.0,0,1,Check-Out,2015-07-03


In [4]:
df.columns

Index(['hotel', 'is_canceled', 'lead_time', 'arrival_date_year',
       'arrival_date_month', 'arrival_date_week_number',
       'arrival_date_day_of_month', 'stays_in_weekend_nights',
       'stays_in_week_nights', 'adults', 'children', 'babies', 'meal',
       'country', 'market_segment', 'distribution_channel',
       'is_repeated_guest', 'previous_cancellations',
       'previous_bookings_not_canceled', 'reserved_room_type',
       'assigned_room_type', 'booking_changes', 'deposit_type', 'agent',
       'company', 'days_in_waiting_list', 'customer_type', 'adr',
       'required_car_parking_spaces', 'total_of_special_requests',
       'reservation_status', 'reservation_status_date'],
      dtype='object')

In [5]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 119390 entries, 0 to 119389
Data columns (total 32 columns):
 #   Column                          Non-Null Count   Dtype  
---  ------                          --------------   -----  
 0   hotel                           119390 non-null  object 
 1   is_canceled                     119390 non-null  int64  
 2   lead_time                       119390 non-null  int64  
 3   arrival_date_year               119390 non-null  int64  
 4   arrival_date_month              119390 non-null  object 
 5   arrival_date_week_number        119390 non-null  int64  
 6   arrival_date_day_of_month       119390 non-null  int64  
 7   stays_in_weekend_nights         119390 non-null  int64  
 8   stays_in_week_nights            119390 non-null  int64  
 9   adults                          119390 non-null  int64  
 10  children                        119386 non-null  float64
 11  babies                          119390 non-null  int64  
 12  meal            

In [6]:
df.is_canceled.value_counts()

is_canceled
0    75166
1    44224
Name: count, dtype: int64

Data doesn't seem extremely imbalanced, so no additional preprocessing for this.

FEATURE ENGINEERING

NAN

In [7]:
count_na = df.isna().sum(axis=0)
count_na[count_na !=0]

children         4
country        488
agent        16340
company     112593
dtype: int64

Categorical Values

Meal

In [8]:
len(df.meal.unique())

5

Countries

In [9]:
len(df.country.unique())

178

Market segment

In [10]:
len(df.market_segment.unique())

8

In [11]:
df.market_segment.value_counts()

market_segment
Online TA        56477
Offline TA/TO    24219
Groups           19811
Direct           12606
Corporate         5295
Complementary      743
Aviation           237
Undefined            2
Name: count, dtype: int64

Distribution Channel

In [12]:
df.distribution_channel.unique()

array(['Direct', 'Corporate', 'TA/TO', 'Undefined', 'GDS'], dtype=object)

In [13]:
df.distribution_channel.value_counts()

distribution_channel
TA/TO        97870
Direct       14645
Corporate     6677
GDS            193
Undefined        5
Name: count, dtype: int64

Reserved room type & Assigned room type

In [14]:
len(df.reserved_room_type.unique())

10

In [15]:
len(df.assigned_room_type.unique())

12

Booking change

In [16]:
df.booking_changes.value_counts()

booking_changes
0     101314
1      12701
2       3805
3        927
4        376
5        118
6         63
7         31
8         17
9          8
10         6
14         5
13         5
15         3
17         2
16         2
12         2
11         2
20         1
21         1
18         1
Name: count, dtype: int64

Outliers. Let's not considered them in the preprocessing

Deposit type

In [17]:
df.deposit_type.value_counts()

deposit_type
No Deposit    104641
Non Refund     14587
Refundable       162
Name: count, dtype: int64

Agent && company

In [18]:
len(df.agent.unique())

334

In [19]:
len(df.company.unique())

353

Customer type

In [20]:
df.customer_type.value_counts()

customer_type
Transient          89613
Transient-Party    25124
Contract            4076
Group                577
Name: count, dtype: int64

In [21]:
df.reservation_status.value_counts()

reservation_status
Check-Out    75166
Canceled     43017
No-Show       1207
Name: count, dtype: int64

Causality

In [22]:
causal_df = df[["is_canceled", "deposit_type", "reservation_status"]]
causal_df.sample(10)

,is_canceled,deposit_type,reservation_status
66796,1,Non Refund,Canceled
25067,0,No Deposit,Check-Out
64479,1,No Deposit,Canceled
5624,1,No Deposit,Canceled
42609,0,No Deposit,Check-Out
105860,0,No Deposit,Check-Out
45642,0,No Deposit,Check-Out
116964,0,No Deposit,Check-Out
14527,0,No Deposit,Check-Out
93292,0,No Deposit,Check-Out


In [23]:
# Check causality between a reservation canceled and Non Refund reservation.
is_canceled_df = causal_df.loc[causal_df["deposit_type"] == "Non Refund"]
is_canceled_df["is_canceled"].sum() / len(is_canceled_df)

np.float64(0.9936244601357374)